# Location-Based Consumer Preference Inference

Notebook used for inspection of the GeoLife dataset, retrieved from  
https://www.microsoft.com/en-us/research/project/geolife-building-social-networks-using-human-location-history/#!downloads  
  
The description of the data can be found in the path `<PATH TO GeoLife>`/Geolife Trajectories 1.3/User Guide-1.3.pdf  

In [11]:
from math import sqrt, cos, sin, pi
import os.path
import sqlite3
from geopy.distance import lonlat, distance, vincenty
import pandas as pd

In [12]:
#### Retrieve data, based on SELECTION from (SQLite) DB stored in DBFILE, located in PATH
def sqlite_2_pd_df(selection, path, dbfile, db):    
    os.chdir(path)
    con=sqlite3.connect(dbfile)
    con.text_factory = sqlite3.OptimizedUnicode
    query = con.execute("SELECT " + selection + " From " + db)
    cols = [column[0] for column in query.description]
    df= pd.DataFrame.from_records(data = query.fetchall(), columns = cols)
    return df

In [13]:
#### Test example
#### The file GeoLifeDBMINI.db holds a SQLite DB with the 
workdir = '/home/patrickm/projects/GeoLife/'
DBfile  = 'GeoLifeDBMINI.db'

In [77]:
#### Create dataframe from SQLite DB
df = sqlite_2_pd_df("*", workdir, DBfile, "GeoLifeDBMINI")

### Unique usrID list
usrIDs = df['usrID'].unique().tolist()
#print (usrIDs)

usrlst = df['usrID'].unique().tolist()
usrlst.sort()

usrdct = {}

for usr in usrlst:
    df[df.usrID == usr ].sort_values(by='trkID', ascending=True)
    trklst = df['trkID'].unique().tolist()
    trklst.sort()
    dct = { trk : trklst.index(trk) for trk in trklst }
#    print (trklst)
#    print(dct)
    usrdct.update( {usr : dct} )
    df['trk no'] = df['trkID'].map(dct)
df = df.set_index(['usrID'])
print(df)

             Lat        Long  Alt       no days    date str  time str  \
usrID                                                                   
0      39.999277  116.324907  492  39775.431863  2008-11-23  10:21:53   
0      39.999335  116.325021  492  39775.431921  2008-11-23  10:21:58   
0      39.999310  116.325017  492  39775.431979  2008-11-23  10:22:03   
0      39.999288  116.325006  492  39775.432037  2008-11-23  10:22:08   
0      39.999275  116.324991  492  39775.432095  2008-11-23  10:22:13   
0      39.999259  116.324952  491  39775.432153  2008-11-23  10:22:18   
0      39.999279  116.324888  492  39775.432211  2008-11-23  10:22:23   
0      39.999309  116.324847  492  39775.432269  2008-11-23  10:22:28   
0      39.999298  116.324857  492  39775.432326  2008-11-23  10:22:33   
0      39.993664  116.326695  -64  39775.436956  2008-11-23  10:29:13   
0      39.993571  116.326662  -92  39775.437014  2008-11-23  10:29:18   
0      39.993507  116.326606 -142  39775.437072  20

In [78]:
#### Function calculating length of a trajectory
def tracklengthGEO(lat, long, alt):
    ln = len(lat)
    trkseg = [ distance((lat[i], long[i], alt[i]), (lat[i-1], long[i-i], alt[i-1] )).miles for i in range(ln) ]
    #trkseg = [ distance.distance((lat[i], long[i], alt[i]), (lat[i-1], long[i-1], alt[i-1])).miles for i in range(ln) ]
    return sum(trkseg)

In [80]:
usr = 0

#print(df.index.name)

#df = df.set_index(['usrID'])
#print(df)
#tracks = df.groupby('trkID')
#print(df)
#for track in
#df.reset_index()

#dft = df.groupby('usrID')#.groupby('trkID')
#dft2= dft.groupby('trkID')

#df.groupby(['usrID','trkID']).size().reset_index().groupby('trkID')[[0]].max()

#print(usrdct[0])

for usr in usrlst:
    tmpdist = []
    tmp = df.loc[usr]
    #print("usr no: ", usr)
    #print(tmp)
    for trk, idx in usrdct[usr].items():
        #print("trk, idx : ", trk, idx)
        main = tmp[tmp['trk no'] == idx ]
        #print(main)
        lat = main['Lat'].tolist()
        long = main['Long'].tolist()
        alt = main['Alt'].tolist()
        tmpdist.append(tracklengthGEO(lat,long,alt))
    print("travel = ", sum(tmpdist))    
#tmp = df.loc[0]
#print(tmp)
#main = tmp[tmp['trk no']==351]

#print("dist = ", tracklengthGEO(lat, long, alt))

travel =  1141088.1210817434
travel =  141646.2845724841
travel =  764595.6944243219
travel =  5876867.8696956225
travel =  2859414.4170113103
travel =  61073.5471516874
travel =  171975.66025857956
travel =  286143.08488445973
travel =  87635.31968278949
